In [1]:
# bibliotecas
import numpy as np

from torch.utils.data import Subset, DataLoader, ConcatDataset, random_split
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, train_test_split
from collections import Counter

In [7]:
import tensorflow as tf

## Escolha da Tarefa de Aprendizado

A tarefa de aprendizado de máquina definida neste trabalho é a classificação de imagens binária, com o objetivo de identificar se uma imagem é real (fotografia de pessoas reais) ou sintética, ou seja, gerada por Inteligência Artificial. Como se trata de um problema visual e a tarefa envolve reconhecer padrões sutis na textura, iluminação e traços que podem diferenciar uma imagem real de uma sintética, optamos por utilizar uma Rede Neural Convolucional (CNN), que é uma arquitetura especialmente eficaz para extração automática de características em tarefas de classificação de imagens.

### Carregamento e Pré-Processamento do Dataset

In [2]:
from utils import data_utils_keras

train_data, test_data = data_utils_keras.load_and_preprocess_data_tf(resize_to=(32, 32))

Found 100000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.


### Divisão Holdout

In [4]:
all_labels = []
for _, labels in train_data:  # train_data é tf.data.Dataset
    all_labels.extend(labels.numpy().tolist())  # converte tensores para lista

label_counts = Counter(all_labels)
print(label_counts)  # ex: Counter({0: 3125, 1: 3125})

Counter({0: 50000, 1: 50000})


In [8]:
# Caminho dos datasets
train_dir = '../data/train'

# Carrega dataset original para poder acessar .class_names depois
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=123
)

# Classes (ordem baseada na alfabética das pastas)
print("Classes:", train_ds.class_names)

Found 100000 files belonging to 2 classes.
Classes: ['FAKE', 'REAL']


O conjunto de dados de treino já está balanceado, contendo 50.000 imagens reais (classe 1) e 50.000 imagens geradas por IA (classe 0). Isso significa que o modelo será treinado com a mesma quantidade de exemplos para cada classe, o que ajuda a evitar viés de aprendizado. Com esse equilíbrio, a CNN tem maiores chances de aprender a distinguir padrões relevantes de cada classe de forma justa, sem favorecer uma categoria em detrimento da outra, o que pode resultar em melhor desempenho e métricas mais confiáveis.


Neste trabalho, optamos por utilizar a validação do tipo holdout estratificado, dividindo o conjunto de dados em 70% para treino, 15% para validação e 15% para teste. Embora a validação cruzada k-fold ofereça uma avaliação mais robusta por repetir o treinamento múltiplas vezes com diferentes divisões dos dados, ela também apresenta um custo computacional significativamente maior, especialmente quando se utiliza CNNs e um volume elevado de dados, como no caso deste projeto. O holdout, por sua vez, permite uma separação mais simples e rápida, com desempenho adequado quando há uma boa quantidade de dados e as classes estão balanceadas. Por isso, a escolha pelo holdout foi feita visando um equilíbrio entre qualidade de avaliação e viabilidade computacional.

In [14]:
all_labels = np.array(all_labels)

split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_idx, val_idx = next(split.split(np.zeros(len(all_labels)), all_labels))

In [52]:
train_split = Subset(train_data, train_idx)
val_split = Subset(train_data, val_idx)

In [30]:
print(f"Treino:    {len(train_split)} imagens")
print(f"Validação: {len(val_split)} imagens")
print(f"Teste:     {len(test_data)} imagens")

Treino:    70000 imagens
Validação: 30000 imagens
Teste:     625 imagens


In [58]:
def get_X_y(dataset):
    X = []
    y = []
    for img, label in dataset:
        X.append(img.numpy())  
        y.append(label)
    X = np.stack(X)
    y = np.array(y)
    return X, y

In [59]:
X_train, y_train = get_X_y(train_split.dataset)
X_val, y_val = get_X_y(val_split.dataset)
X_test, y_test = get_X_y(test_data)

### Métricas 

Para a avaliação de desempenho do modelo foi escolhido as métricas: acurácia, F1-score macro, matriz de confusão e curva de aprendizado.

- Acurácia

Uma métrica fundamental para avaliar modelos de classificação, especialmente quando as classes estão balanceadas, como é o caso do dataset utilizado. 

- F1-Score Macro 

É importante pois ele calcula a média do F1-Score para cada classe, tratando todas com igual importância, também leva em conta tanto a precisão quanto a revocação, sendo mais robusto em problemas de classificação binária.

- Matriz de confusão 

É essencial para entender como o modelo está errando, mostrando os falsos positivos e falsos negativos. Além disso, permite uma análise mais profunda do comportamento do modelo em relação a cada classe.

- Curva de aprendizado 

É um recurso visual que ajuda a diagnosticar problemas de overfitting ou underfitting. Monitorar a evolução da loss e da acurácia no conjunto de treinamento e validação ao longo das épocas é uma prática essencial no desenvolvimento de CNNs.

- AUC-ROC

A Área sob a Curva ROC é uma métrica que avalia a capacidade do modelo em distinguir entre as classes. Um valor de AUC próximo de 1 indica uma excelente capacidade discriminativa, enquanto um valor próximo de 0.5 sugere que o modelo está performando de forma semelhante ao acaso.